In [4]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import tqdm
import pyvista as pv
from scipy.spatial import cKDTree
from suncalc import get_position
from pybdshadow.analysis import get_timetable
from pybdshadow.utils import make_clockwise
from pyproj import Transformer

In [ ]:
def convert_to_utm50n(lon, lat):
    """
    将WGS84经纬度坐标转换为UTM50N坐标系
    
    Parameters
    ----------
    lon, lat : float
        经度和纬度(WGS84)
        
    Returns
    -------
    tuple
        (easting, northing) UTM50N坐标(米)
    """
    # 创建从WGS84到UTM50N的转换器
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:32650", always_xy=True)
    
    # 执行转换
    easting, northing = transformer.transform(lon, lat)
    return easting, northing

def convert_to_model_coords_utm50n(lon, lat, lon_center, lat_center):
    """
    将GPS坐标转换为以指定中心点为原点的UTM50N模型坐标系
    
    Parameters
    ----------
    lon, lat : float
        目标点的经度和纬度
    lon_center, lat_center : float
        中心点的经度和纬度
        
    Returns
    -------
    tuple
        (x, y) 相对于中心点的坐标(米)
    """
    # 获取中心点的UTM坐标
    center_easting, center_northing = convert_to_utm50n(lon_center, lat_center)
    
    # 获取目标点的UTM坐标
    point_easting, point_northing = convert_to_utm50n(lon, lat)
    
    # 计算相对于中心点的偏移
    x = point_easting - center_easting
    y = point_northing - center_northing
    
    return x, y

def get_study_area_bounds(building_mesh, buffer=500):
    """
    获取研究区域的边界
    
    Parameters
    ----------
    building_mesh : pyvista.PolyData
        建筑物3D模型
    buffer : float
        边界缓冲区(米)
        
    Returns
    -------
    tuple
        (x_min, x_max, y_min, y_max)
    """
    bounds = building_mesh.bounds
    x_min, x_max = bounds[0] - buffer, bounds[1] + buffer
    y_min, y_max = bounds[2] - buffer, bounds[3] + buffer
    return x_min, x_max, y_min, y_max

def get_solar_data_for_time(datetime):
    """
    获取特定时间的太阳辐射数据
    
    Parameters
    ----------
    datetime : pandas.Timestamp
        查询的时间点
        
    Returns
    -------
    pandas.Series
        包含GHI, DNI, DHI, Temperature, Wind Speed的数据行
    """
    # 读取太阳辐射数据
    solar_irradiance_path = 'solar/solar_irradiance.csv'
    solar_data = pd.read_csv(solar_irradiance_path)
    solar_data['datetime'] = pd.to_datetime(solar_data[['Year', 'Month', 'Day', 'Hour', 'Minute']])
    
    # 查找匹配的数据行
    solar_record = solar_data[solar_data['datetime'] == datetime]
    return solar_record

def get_sundir(date, lon, lat):
    """
    计算太阳方向向量
    
    Parameters
    ----------
    date : datetime
        日期时间
    lon, lat : float
        位置的经纬度
        
    Returns
    -------
    tuple
        (sun_direction, sun_position)
    """
    # 获取太阳位置
    sunPosition = get_position(date, lon, lat)
    azimuth = sunPosition['azimuth']
    altitude = sunPosition['altitude']

    # 将角度转换为弧度
    azimuth_rad = -azimuth + np.pi/2
    altitude_rad = altitude

    # 计算球坐标系中的向量分量
    x = np.cos(altitude_rad) * np.cos(azimuth_rad)
    y = np.cos(altitude_rad) * np.sin(azimuth_rad)
    z = np.sin(altitude_rad)

    # 太阳光的方向是从太阳指向地球，因此需要反转向量
    sun_direction = np.array([x, y, -z])

    return sun_direction, sunPosition

def precompute_radiation_field(building_mesh, lon_center, lat_center, dates, resolution=10):
    """
    预计算研究区域的辐射场
    
    Parameters
    ----------
    building_mesh : pyvista.PolyData
        建筑物3D模型
    lon_center, lat_center : float
        研究区域中心坐标
    dates : list
        需要计算的日期列表
    resolution : float
        空间分辨率(米)
    
    Returns
    -------
    dict
        包含所有时间点辐射场数据的字典
    """
    print("Precomputing radiation field...")
    
    # 获取研究区域范围
    x_min, x_max, y_min, y_max = get_study_area_bounds(building_mesh)
    
    # 创建网格点
    x_range = np.arange(x_min, x_max, resolution)
    y_range = np.arange(y_min, y_max, resolution)
    
    print(f"Creating grid with {len(x_range)}x{len(y_range)} points...")
    grid_points = np.array([(x, y, 1.0) for x in x_range for y in y_range])  # 高度设为1米(车顶高度)
    
    # 获取时间表
    date_times = get_timetable(lon_center, lat_center, dates=dates, precision=600, padding=0)  # 10分钟精度
    
    # 创建空间索引（使用scipy的KDTree）- 只需创建一次
    spatial_index = cKDTree(grid_points[:, :2])  # 只用x,y坐标建立索引
    
    # 初始化辐射数据结构
    radiation_data = {
        'grid_points': grid_points,
        'spatial_index': spatial_index,
        'time_points': [],
        'radiation_values': {},
        'temperature': {},
        'wind_speed': {},
        'dni': {},
        'dhi': {},
        'sun_position': {}
    }
    
    # 为每个时间点计算辐射场
    for current_time in tqdm.tqdm(date_times['datetime'], desc="Computing radiation field"):
        # 计算太阳方向及位置
        sun_direction, sun_position = get_sundir(current_time, lon_center, lat_center)
        
        # 如果太阳在地平线以下，跳过
        if sun_position['altitude'] <= 0:
            continue
            
        # 获取辐射数据
        rounded_time = pd.to_datetime(current_time).round('10min')
        rounded_time_str = rounded_time.strftime('%Y-%m-%d %H:%M:%S')
        
        solar_record = get_solar_data_for_time(rounded_time)
        if solar_record.empty:
            continue
            
        # 批量光线追踪计算遮阳
        ray_origins = grid_points - sun_direction * 5e5  # 光线起点
        ray_directions = np.tile(sun_direction, (len(ray_origins), 1))  # 光线方向
        
        # 执行批量光线追踪
        intersection_points, intersection_rays, _ = building_mesh.multi_ray_trace(
            ray_origins, ray_directions, first_point=True)
        
        # 创建未被遮挡的点索引集合
        unshaded_indices = set(range(len(grid_points))) - set(intersection_rays)
        
        # 计算辐射量并存储
        radiation_values = np.zeros(len(grid_points))
        
        # 基础辐射值
        ghi = solar_record['GHI'].iloc[0]
        dni = solar_record['DNI'].iloc[0]
        dhi = solar_record['DHI'].iloc[0]
        
        # 为未遮挡的点计算辐射
        for idx in unshaded_indices:
            # 直射部分考虑太阳高度角，散射部分不变
            radiation_values[idx] = dni * np.sin(sun_position['altitude']) + dhi
        
        # 存储当前时间点的数据
        radiation_data['time_points'].append(rounded_time_str)
        radiation_data['radiation_values'][rounded_time_str] = radiation_values
        radiation_data['temperature'][rounded_time_str] = solar_record['Temperature'].iloc[0]
        radiation_data['wind_speed'][rounded_time_str] = solar_record['Wind Speed'].iloc[0]
        radiation_data['dni'][rounded_time_str] = dni
        radiation_data['dhi'][rounded_time_str] = dhi
        radiation_data['sun_position'][rounded_time_str] = sun_position
    
    print(f"Radiation field computed for {len(radiation_data['time_points'])} time points")
    return radiation_data

def save_radiation_data(radiation_data, output_path):
    """
    保存预计算的辐射场数据
    
    Parameters
    ----------
    radiation_data : dict
        辐射场数据
    output_path : str
        输出文件路径
    """
    # 创建一个可序列化的数据结构
    serializable_data = {
        'grid_points': radiation_data['grid_points'].tolist(),
        'time_points': radiation_data['time_points'],
        'radiation_values': {k: v.tolist() for k, v in radiation_data['radiation_values'].items()},
        'temperature': radiation_data['temperature'],
        'wind_speed': radiation_data['wind_speed'],
        'dni': radiation_data['dni'],
        'dhi': radiation_data['dhi'],
        'sun_position': radiation_data['sun_position']
    }
    
    # 保存为numpy压缩文件
    np.savez_compressed(output_path, data=serializable_data)
    print(f"Radiation data saved to {output_path}")

def load_radiation_data(input_path):
    """
    加载预计算的辐射场数据
    
    Parameters
    ----------
    input_path : str
        输入文件路径
        
    Returns
    -------
    dict
        辐射场数据
    """
    # 加载压缩文件
    loaded = np.load(input_path, allow_pickle=True)
    data = loaded['data'].item()
    
    # 重建数据结构
    radiation_data = {
        'grid_points': np.array(data['grid_points']),
        'time_points': data['time_points'],
        'radiation_values': {k: np.array(v) for k, v in data['radiation_values'].items()},
        'temperature': data['temperature'],
        'wind_speed': data['wind_speed'],
        'dni': data['dni'],
        'dhi': data['dhi'],
        'sun_position': data['sun_position']
    }
    
    # 重建空间索引
    radiation_data['spatial_index'] = cKDTree(radiation_data['grid_points'][:, :2])
    
    print(f"Loaded radiation data with {len(radiation_data['time_points'])} time points")
    return radiation_data

def process_file_precomputed(file_path, radiation_data, lon_center, lat_center):
    """
    处理轨迹数据并计算每条轨迹的光伏发电量
    
    Parameters
    ----------
    file_path : str
        轨迹数据CSV路径
    radiation_data : dict
        预计算的辐射场数据
    lon_center, lat_center : float
        研究区域中心坐标
        
    Returns
    -------
    pandas.DataFrame
        处理后的轨迹数据
    """
    print(f"Processing trajectory file: {file_path}")
    
    # 读取轨迹数据
    df = pd.read_csv(file_path)
    
    # 将日期时间列转换为datetime
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # 提取日期列，保持和输入格式一致
    df['date_str'] = df['date']  # 保留原始date字符串
    
    # 计算时间间隔 (以秒为单位)
    df = df.sort_values(['ID', 'datetime'])
    df['deltat'] = df.groupby('ID')['datetime'].diff().dt.total_seconds().fillna(0)
    
    # 将时间四舍五入到10分钟
    df['rounded_datetime'] = df['datetime'].dt.round('10min')
    
    # 光伏板参数
    panel_area = 2.0  # 车顶光伏面积(平方米)
    panel_efficiency = 0.22  # 光伏转换效率
    
    # 初始化结果列
    df['shined'] = 0
    df['calculated_irradiance'] = 0
    df['calculated_temperatures'] = np.nan
    
    # 获取空间索引
    spatial_index = radiation_data['spatial_index']
    grid_points = radiation_data['grid_points']
    
    # 对每条轨迹进行处理
    for idx, row in tqdm.tqdm(df.iterrows(), total=len(df), desc="Processing trajectories"):
        # 将时间戳转换为字符串格式以匹配辐射数据中的键
        time_str = row['rounded_datetime'].strftime('%Y-%m-%d %H:%M:%S')
        
        # 检查时间点是否在辐射数据中
        if time_str not in radiation_data['time_points']:
            continue
        
        # 获取车辆位置并转换到UTM50N模型坐标系
        vehicle_x, vehicle_y = convert_to_model_coords_utm50n(row['lng'], row['lat'], lon_center, lat_center)
        
        # 查找最近的网格点
        dist, nearest_idx = spatial_index.query([vehicle_x, vehicle_y])
        
        # 获取该点的辐射值
        irradiance = radiation_data['radiation_values'][time_str][nearest_idx]
        
        if irradiance > 0:
            # 考虑车辆朝向对辐射的影响
            sun_position = radiation_data['sun_position'][time_str]
            azimuth = sun_position['azimuth']
            altitude = sun_position['altitude']
            
            angle_rad = np.radians(row['angle'])
            
            # 计算太阳向量在水平面上的投影
            sun_vector_horizontal = np.array([
                np.cos(altitude) * np.sin(azimuth),
                np.cos(altitude) * np.cos(azimuth)
            ])
            
            # 车辆朝向向量
            car_vector = np.array([np.sin(angle_rad), np.cos(angle_rad)])
            
            # 计算水平面上的夹角余弦值
            cos_angle_horizontal = np.dot(sun_vector_horizontal, car_vector) / (
                np.linalg.norm(sun_vector_horizontal) * np.linalg.norm(car_vector))
            
            # 考虑太阳高度角和水平夹角的综合影响
            irradiance_factor = np.sin(altitude) * (0.5 + 0.5 * cos_angle_horizontal)
            
            # 分解辐射为直射和散射
            dni = radiation_data['dni'][time_str]
            dhi = radiation_data['dhi'][time_str]
            
            # 直射部分考虑角度，散射部分不变
            effective_dni = dni * irradiance_factor
            effective_irradiance = effective_dni + dhi
            
            # 标记为非遮挡并记录辐射值
            df.loc[idx, 'shined'] = 1
            df.loc[idx, 'calculated_irradiance'] = effective_irradiance
            
            # 计算电池温度
            ambient_temp = radiation_data['temperature'][time_str]
            wind_speed = radiation_data['wind_speed'][time_str]
            
            a = -3.56
            b = -0.075
            delta_T = 3
            cell_temp = effective_irradiance * np.exp(a + b * wind_speed) + ambient_temp + delta_T
            df.loc[idx, 'calculated_temperatures'] = cell_temp
    
    # 计算发电量
    print("Calculating power generation...")
    
    # 温度系数
    gamma_pdc = -0.004
    temp_ref = 25  # 参考温度(°C)
    
    # 计算温度修正后的效率和DC功率
    df['adjusted_efficiency'] = panel_efficiency * (1 + gamma_pdc * (df['calculated_temperatures'] - temp_ref))
    df['dc_power'] = df['calculated_irradiance'] * panel_area * df['adjusted_efficiency']
    df['dc_power'] = df['dc_power'].fillna(0).clip(lower=0)
    
    # 考虑逆变器效率
    inverter_efficiency = 0.96
    df['ac_power'] = df['dc_power'] * inverter_efficiency
    
    # 计算每个时间段的能量(kWh)
    df['power'] = df.apply(lambda row: (row['ac_power']/1000) * (row['deltat']/3600), axis=1)
    
    # 按日期聚合发电量
    df['date'] = df['datetime'].dt.date
    daily_generation = df.groupby('date')['power'].sum().reset_index()
    daily_generation.rename(columns={'power': 'daily_energy_kwh'}, inplace=True)
    
    # 将日发电量合并回原始数据框
    df = df.merge(daily_generation, on='date')
    
    return df


def process_all_files_with_precomputed_radiation(building_mesh, lon_center, lat_center, input_directory, output_directory, radiation_data_path=None):
    """
    使用预计算的辐射场处理所有轨迹文件
    
    Parameters
    ----------
    building_mesh : pyvista.PolyData
        建筑物3D模型
    lon_center, lat_center : float
        研究区域中心坐标
    input_directory : str
        输入轨迹文件目录
    output_directory : str
        输出结果文件目录
    radiation_data_path : str, optional
        辐射场数据文件路径，如果提供则加载已有数据，否则重新计算
    """
    # 确保输出目录存在
    os.makedirs(output_directory, exist_ok=True)
    
    # 获取所有轨迹文件
    trajectory_files = [f for f in os.listdir(input_directory) if f.endswith('.csv')]
    
    if not trajectory_files:
        print(f"No CSV files found in {input_directory}")
        return
    
    # 决定是加载还是计算辐射场数据
    if radiation_data_path and os.path.exists(radiation_data_path):
        print(f"Loading precomputed radiation data from {radiation_data_path}")
        radiation_data = load_radiation_data(radiation_data_path)
    else:
        # 从轨迹文件中获取日期范围
        all_dates = []
        for file in trajectory_files:
            df = pd.read_csv(os.path.join(input_directory, file))
            df['datetime'] = pd.to_datetime(df['datetime'])  # 使用datetime列
            dates = df['datetime'].dt.date.unique()
            all_dates.extend([str(date) for date in dates])
        
        # 去重
        unique_dates = list(set(all_dates))
        
        # 预计算所有日期的辐射场
        radiation_data = precompute_radiation_field(building_mesh, lon_center, lat_center, unique_dates)
        
        # 如果提供了路径，保存辐射场数据以便将来使用
        if radiation_data_path:
            save_radiation_data(radiation_data, radiation_data_path)
    
    # 处理每个轨迹文件
    for file in trajectory_files:
        input_path = os.path.join(input_directory, file)
        df_result = process_file_precomputed(input_path, radiation_data, lon_center, lat_center)
        
        # 保存结果
        output_path = os.path.join(output_directory, file)
        df_result.to_csv(output_path, index=False)
        print(f"Results saved to {output_path}")
    
    # 汇总所有车辆的日发电量
    all_daily_data = []
    for file in trajectory_files:
        output_path = os.path.join(output_directory, file)
        df = pd.read_csv(output_path)
        df['vehicle_id'] = file.split('.')[0]  # 假设文件名是车辆ID
        daily_data = df[['date', 'vehicle_id', 'daily_energy_kwh']].drop_duplicates()
        all_daily_data.append(daily_data)
    
    if all_daily_data:
        # 合并所有车辆的日发电量
        all_daily_df = pd.concat(all_daily_data)
        
        # 按日期和车辆聚合
        daily_summary = all_daily_df.groupby(['date']).agg({
            'daily_energy_kwh': 'sum',
            'vehicle_id': 'count'
        }).reset_index()
        daily_summary.rename(columns={'vehicle_id': 'vehicle_count'}, inplace=True)
        
        # 保存日发电量汇总
        daily_summary_path = os.path.join(output_directory, 'daily_generation_summary.csv')
        daily_summary.to_csv(daily_summary_path, index=False)
        print(f"Daily generation summary saved to {daily_summary_path}")
    
    print("All files processed successfully.")

def main():
    # 加载建筑物数据并生成3D模型
    building_data_path = 'shenzhen_building.geojson' 
    building_data = gpd.read_file(building_data_path)
    
    # 生成建筑物3D模型
    building_mesh = generate_mesh(building_data)
    
    # 研究区域中心坐标
    lon_center = 114.057868
    lat_center = 22.543099
    
    # 输入输出目录
    input_directory = 'traj/'
    output_directory = 'output/'
    
    # 辐射场数据路径
    radiation_data_path = 'radiation_field/radiation_field.npz'
    
    # 处理所有文件
    process_all_files_with_precomputed_radiation(
        building_mesh, 
        lon_center, 
        lat_center, 
        input_directory, 
        output_directory,
        radiation_data_path
    )


In [6]:
if __name__ == "__main__":
    main()

Generating 3D mesh:   0%|          | 608/437390 [00:05<1:06:29, 109.49it/s]


NotAllTrianglesError: Input mesh for subdivision must be all triangles.